## Check

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URLの設定
url = 'http://www1.river.go.jp/cgi-bin/DspWaterData.exe?KIND=2&ID=307111287708010&BGNDATE=20241101&ENDDATE=20241231&KAWABOU=NO'

# HTMLコンテンツの取得
response = requests.get(url)

# BeautifulSoupでHTMLを解析
soup = BeautifulSoup(response.content, 'html.parser')

# pandasでHTMLテーブルを抽出
tables = pd.read_html(str(soup))

# 最初のテーブルを取得（インデックスを0に指定）
df = tables[1]

# 結果を表示


# print(df)

/tmp/ipykernel_3874574/273463336.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup))


In [17]:
import pandas as pd

# 最初の2行をヘッダーとして再設定
df.columns = df.iloc[1]

# 最初の2行と最後の1行を削除
df = df.drop([0, 1, len(df) - 1])

# インデックスをリセット
df = df.reset_index(drop=True)

# 結果を確認
print(df)

1          NaN    1時    2時    3時    4時    5時    6時    7時    8時    9時  ...  \
0   2016/01/01  0.55  0.55  0.55  0.55  0.55  0.55  0.53  0.49  0.46  ...   
1   2016/01/02  0.53  0.54  0.53  0.54  0.54  0.54  0.52  0.49  0.46  ...   
2   2016/01/03  0.53  0.53  0.53  0.53  0.52  0.49  0.45  0.42  0.40  ...   
3   2016/01/04  0.53  0.53  0.52  0.49  0.45  0.42  0.40  0.38  0.37  ...   
4   2016/01/05  0.52  0.52  0.53  0.53  0.53  0.53  0.52  0.49  0.45  ...   
5   2016/01/06  0.52  0.52  0.52  0.52  0.52  0.52  0.51  0.49  0.45  ...   
6   2016/01/07  0.53  0.53  0.53  0.53  0.52  0.49  0.45  0.42  0.40  ...   
7   2016/01/08  0.53  0.53  0.53  0.53  0.52  0.49  0.45  0.42  0.40  ...   
8   2016/01/09  0.53  0.53  0.53  0.53  0.52  0.49  0.45  0.42  0.40  ...   
9   2016/01/10  0.53  0.53  0.52  0.49  0.45  0.42  0.40  0.38  0.37  ...   
10  2016/01/11  0.40  0.38  0.37  0.36  0.36  0.36  0.36  0.36  0.36  ...   
11  2016/01/12  0.39  0.37  0.36  0.35  0.35  0.35  0.35  0.35  0.35  ...   

## 以下，関数を用いて作ろうとしたプログラム

In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# データ取得関数
def fetch_data(month, year):
    # URL設定
    start_date = f"{year}{month:02d}01"  # BGNDATE（開始日）
    # 終了日は月の最終日を計算
    if month == 12:
        end_date = f"{year + 1}0101"  # 次の年の1月1日を終了日
    else:
        end_date = f"{year}{month + 1:02d}01"  # 次の月の1日を終了日
    url = f'http://www1.river.go.jp/cgi-bin/DspWaterData.exe?KIND=2&ID=307111287708010&BGNDATE={start_date}&ENDDATE={end_date}&KAWABOU=NO'
    
    # HTMLコンテンツの取得
    response = requests.get(url)
    
    # BeautifulSoupでHTMLを解析
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # pandasでHTMLテーブルを抽出
    tables = pd.read_html(str(soup))
    
    # 最初のテーブルを取得
    df = tables[0]
    print(df)
    
    # 必要な列（仮定: 時刻と水位）
    df = df[['時刻', '水位']]
    
    # '時刻'列をpandasのdatetime形式に変換
    df['時刻'] = pd.to_datetime(df['時刻'], format='%Y/%m/%d %H:%M')
    
    # '水位'列の空白や欠損部分をnullに変換
    df['水位'] = pd.to_numeric(df['水位'], errors='coerce')
    
    # 時間インデックスを設定
    df.set_index('時刻', inplace=True)
    
    # 欠損値をnullにする処理
    df = df.where(pd.notnull(df), None)
    
    # 1時間単位でデータをリサンプリング（欠損値はそのままnullに）
    df_resampled = df.resample('H').asfreq()
    
    return df_resampled

# 2016年1月から現在の月までのデータを取得
start_year = 2016
end_year = 2024
current_month = 12  # 例: 現在の月

all_data = pd.DataFrame()

# 各月ごとにデータを取得
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        if year == end_year and month > current_month:
            break
        print(f"Fetching data for {year}-{month:02d}...")
        monthly_data = fetch_data(month, year)
        all_data = pd.concat([all_data, monthly_data])

# 結果の確認
print(all_data)

Fetching data for 2016-01...
                 0         1    2    3
0            観測所記号      観測所名  水系名  河川名
1  307111287708010  高梁（たかはし）  高梁川  高梁川


/tmp/ipykernel_3874574/3827931642.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup))


KeyError: "None of [Index(['時刻', '水位'], dtype='object')] are in the [columns]"